# <span style="color: blue;">Model creation using Google Bert</span>
#### <span style="color: gray;"> Reference - Below link was taken as a reference to come up with the working model for tweet data.</span>
##### https://towardsdatascience.com/sentiment-analysis-in-10-minutes-with-bert-and-hugging-face-294e8a04b671

# <span style="color: green;">Imports needed for Tensor flow to work</span>

In [18]:
from IPython.display import HTML
HTML('''<script>
    code_show_err=false; 
    function code_toggle_err() {
     if (code_show_err){
     $('div.output_stderr').hide();
     } else {
     $('div.output_stderr').show();
     }
     code_show_err = !code_show_err
    } 
    $( document ).ready(code_toggle_err);
    </script>
    To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

In [19]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import time
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import nltk
import re

In [20]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import word_tokenize
from imblearn.over_sampling import SMOTE

# <span style="color: green;">Imports needed for Tensor flow to work</span>

In [21]:
#from ipynb.fs.full.function import stemmer
#from ipynb.fs.full.function import lemmetizer
from ipynb.fs.full.function import print_model_data
from ipynb.fs.full.function import model_execute
from ipynb.fs.full.function import stemm_data
from ipynb.fs.full.function import lemm_data
from ipynb.fs.full.function import class_balancer
from ipynb.fs.full.function import bar_plot
from ipynb.fs.full.function import get_data
from ipynb.fs.full.function import remove_punctuation_func
from ipynb.fs.full.function import convert_data_to_examples
from ipynb.fs.full.function import convert_examples_to_tf_dataset
from ipynb.fs.full.function import get_live_tweets

# <span style="color: green;">Creating the model and tokenizer for Google Bert</span>

In [22]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_75 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


# <span style="color: green;"> Prepare the text for Google Bert to process the data into Bert Data Set. </span>
# <span style="color: green;"> Here the text column is loaded iteratively and saved in a new column replaced with a ', ' </span>
# <span style="color: green;">   at the end to mark end of one entry</span>

In [24]:
tweets = pd.read_csv('data/train.csv')
tweets
replaced = []
text = tweets['text']

# for i in text:
#     replaced.append(i+", ")
    
# tweets['replaced'] = replaced
tweets = tweets.drop('keyword',axis=1).drop('location',axis=1)

tweets

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,"13,000 people receive #wildfires evacuation orders in California",1
4,7,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1
...,...,...,...
7608,10869,Two giant cranes holding a bridge collapse into nearby homes http://t.co/STfMbbZFB5,1
7609,10870,@aria_ahrary @TheTawniest The out of control wild fires in California even in the Northern part of the state. Very troubling.,1
7610,10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ,1
7611,10872,Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.,1


# <span style="color: green;"> Creating a postive Data Set for Google Bert</span>

In [25]:
positives=''
positives = tweets.query("`target`==1")
positives

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,"13,000 people receive #wildfires evacuation orders in California",1
4,7,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1
...,...,...,...
7608,10869,Two giant cranes holding a bridge collapse into nearby homes http://t.co/STfMbbZFB5,1
7609,10870,@aria_ahrary @TheTawniest The out of control wild fires in California even in the Northern part of the state. Very troubling.,1
7610,10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ,1
7611,10872,Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.,1


# <span style="color: green;"> Creating a negative Data Set for Google Bert </span>

In [26]:
negavites=''
negatives = tweets.query("`target`==0")
negatives

,id,text,target
15,23,What's up man?,0
16,24,I love fruits,0
17,25,Summer is lovely,0
18,26,My car is so fast,0
19,28,What a goooooooaaaaaal!!!!!!,0
...,...,...,...
7581,10833,@engineshed Great atmosphere at the British Lion gig tonight. Hearing is wrecked. http://t.co/oMNBAtJEAO,0
7582,10834,Cramer: Iger's 3 words that wrecked Disney's stock - CNBC http://t.co/N6RBnHMTD4,0
7584,10837,These boxes are ready to explode! Exploding Kittens finally arrived! gameofkittens #explodingkittensÛ_ https://t.co/TFGrAyuDC5,0
7587,10841,Sirens everywhere!,0


# <span style="color: green;"> Save the replaced column into a file to feed it into Google Bert</span>

In [27]:
count=0
for i in positives['text']:
    filename='data/train/pos/pos'+str(count)+'.txt'
    finalArray=np.array([i])
    np.savetxt(filename, finalArray, fmt='%s')
    count=count+1
count=0
for j in negatives['text']:
    filename='data/train/neg/neg'+str(count)+'.txt'
    finalArray=np.array([j])
    np.savetxt(filename, finalArray, fmt='%s')
    count=count+1

# <span style="color: green;"> Google Bert needs a specific folder structure.</span>
# <span style="color: green;"> 1. Top level folder which is fed to the tensor flow pre processing </span>
# <span style="color: green;"> 2. multiple files for each of the class, in this case Postive and Negative. </span>
# <span style="color: green;"> Here a manual step of creating this folder structure is performed. </span>

In [28]:
train = tf.keras.preprocessing.text_dataset_from_directory('data/',batch_size=30000, validation_split=0.2, 
   subset='training', seed=123)

print(train.take(1))
test = tf.keras.preprocessing.text_dataset_from_directory('data/', batch_size=30000, validation_split=0.2, 
    subset='validation', seed=123)

Found 7614 files belonging to 2 classes.
Using 6092 files for training.
<TakeDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
Found 7614 files belonging to 2 classes.
Using 1522 files for validation.


# <span style="color: green;"> Create a DataFrame from the Bert BatchDataSet</span>

In [29]:
train_feat=''
train_lab=''
for i in train.take(2):
    train_feat = i[0].numpy()
    train_lab = i[1].numpy()

train = pd.DataFrame([train_feat, train_lab]).T
train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
train['DATA_COLUMN'] = train['DATA_COLUMN'].str.decode("utf-8")
pd.set_option('display.max_colwidth', None)
train.head()

,DATA_COLUMN,LABEL_COLUMN
0,Fire in Pisgah National Forest grows to 375 acres http://t.co/dao9AZEUcr\n,1
1,Tsunami - DVBBS &amp; Borgeous (Arceen Festival Trap Remix) https://t.co/743JoqazrT via @YouTube\n,1
2,Ladies here's how to recover from a #date you totally BOMBED... according to men http://t.co/c5GGSZUGw1 http://t.co/2PiMg9BIcE\n,1
3,@mattcohen4fake Gamma Ray January Worlds Collide She Waits Be Me Wave Past Perfect Reunion Lucky Cool If I Come Over Hot Times...\n,1
4,Spokane authorities say they're struggling to solve arson cases like today's on Hamilton. http://t.co/Qbs2k01WzK http://t.co/mvLZIYsGLL\n,1


# <span style="color: green;"> Create the Test data in a similar way for our validation </span>

In [30]:
for j in test.take(2):
    test_feat = j[0].numpy()
    test_lab = j[1].numpy()

test = pd.DataFrame([test_feat, test_lab]).T
test.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
test['DATA_COLUMN'] = test['DATA_COLUMN'].str.decode("utf-8")
test.head()

,DATA_COLUMN,LABEL_COLUMN
0,Lunch for the crew is made. Night night it's been a long day! \n~Peace~Love~Rescue~\n,1
1,WSJThinkTank: Ahead of tonight's #GOPDebate ColleenMNelson explains how a bad debate can derail a campaign: Û_ http://t.co/XyxTuACZvb\n,1
2,y'all read 12000 Nigerian refugees repatriated from Cameroon http://t.co/aVwE1LBvhn\n,1
3,Aspiring musician &amp; song writer shares her talent at the GMMBC Youth Explosion on this past Saturday. http://t.co/OmjMTU9kFG\n,1
4,The Latest: More homes razed by Northern California wildfire - http://t.co/R1CNSjUAYQ http://t.co/DQ1yLcrF9K\n,1


# <span style="color: green;"> Generate the Training and Validation Data </span>

In [31]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, 'DATA_COLUMN', 'LABEL_COLUMN')

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/Users/achandrasekhar/opt/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2323: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [32]:
print(train_data)

<RepeatDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


# <span style="color: green;"> We will use Adam as our optimizer, CategoricalCrossentropy as our loss function, and SparseCategoricalAccuracy as our accuracy metric. Fine-tuning the model for 2 epochs will give us around 95% accuracy</span>



In [33]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])


model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
382/382 [==============================] - 3501s 9s/step - loss: 0.0121 - accuracy: 0.9966 - val_loss: 1.4649e-04 - val_accuracy: 1.0000
Epoch 2/2
382/382 [==============================] - 3416s 9s/step - loss: 8.2430e-04 - accuracy: 0.9998 - val_loss: 5.6582e-05 - val_accuracy: 1.0000


In [66]:
sentences = []
live_tweets=get_live_tweets()
for tweet in live_tweets['tweets']:
    sentences.append(tweet)

# <span style="color: green;"> Use the model for predicting the sample data.</span>

In [67]:
preds=[]
tf_batch = tokenizer(sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['0','1']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(live_tweets)):
    preds.append(list(labels[label[i]]))
live_tweets['preds']=preds
live_tweets

,tweets,preds
0,RT @moonlitmuses: no but who gonna tell these two to they're setting the screen on fire with this act of theirs already 🔥🤌🏻\n#Yrkkh \nhttps:/…,[1]
1,@planetjedward What so that you can be ruled by the EU ?\nOut of the frying pan and into the fire 🤷‍♂️,[1]
2,"GIRtech INSA-2 Insulation Fire Brick, 2-Pack, for Kilns, Forges, Metal Clay Firing, Jewelry Soldering, 2370 °F Rated [ETFCUQN]\n\nhttps://t.co/gRNuLn4Eea",[1]
3,RT @UltraLandlord: Another electric bike caught fire during charging. \n\nThe worst part about burning lithium batteries is that you can't pu…,[1]
4,RT @TheJokerBhai: Listen and Learn \nHope Everybody has seen this #SushantSinghRajput𓃵 quick fire questions \nBut Still Culprits Roam Free I…,[1]
5,"Clarify through fight fire, what do you define success, you must\nAn empire ـ⁦فاشونـ there isn't ے⁦ماكس⁩- . The only loss ك̷و̷د̶⁩ ⁦̸خ̷ص̸م̴⁩ . is the best part https://t.co/xegRixks00",[1]
6,RT @neknekmo1819: Fuel To The Fire: A #SeKen AU\n\nKen is a former drag racing champion who met an accident which marred his dreams of racing…,[1]
7,RT @SanFranMuso: @City_Press .@AngieMotshekga @DBE_SA you and your department are bloody useless. You have broken the schooling system and…,[1]
8,RT @The_Gator1: @jay_fcb @MetaBeatOffl MetaBeat will spread like wild fire Even the blind will see MetBeat and hop on 🚀🚀\n#MetaBeat \n#METABE…,[1]
9,THIS IS MY ULTIMATE and A clear version acapella of Namjoon's fire rapping is just 🤯🤯🤯🤯🤯🤯\n\n#NAMJOONDAY #OurEternalLeader #Namjoonthebestrapper https://t.co/yDUc8QdB8z,[1]
